# 3D Protein Reconstruction using Alpha Fold Protein Data bank (PDB) structures

We begin by using py3Dmol, a python library to visualize what the PDB file looks like. The following snippet will allow us to see what the protein structure looks like.

In [ ]:
import py3Dmol

pdb_file = "input/AF-O06917-F1-model_v4.pdb"

with open(pdb_file, "r") as f:
    pdb_data = f.read()

view = py3Dmol.view(width=800, height=600)

view.addModel(pdb_data, "pdb")

view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()

view.show()

Now, we'll follow these steps to reconstruct the protein:
1. We first generate 3 random angles or views of the protein and store these coordinates.
2. We vertically stack these three views together and train a machine learning model to learn the mapping between the input views and the original protein structure.
3. We then use the trained model to generate the reconstructed coordinates for the protein structure.
5. Lastly, we create a visualization using point clouds using Plotly to see how closely it resembles the original protein.

Let's begin with our import statements

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np
import plotly.graph_objects as go

Now, we'll write a method to parse the PDB files and grab the coordinates associated with each atom.

In [ ]:
def parse_pdb(file_path):
    coordinates = []
    with open(file_path, 'r') as pdb_file:
        for line in pdb_file:
            if line.startswith('ATOM'):
                x, y, z = float(line[30:38]), float(line[38:46]), float(line[46:54])
                coordinates.append([x, y, z])
    return np.array(coordinates)

Once we've parsed the PDB file, we need to generate the random views. In order to do this, we'll have one view be the original coordinates, the second view be some random view of the coordinates using numpy.random, and the third view will be a flip of the original coordinates. The goal of this is to have 

In [ ]:
def generate_views(coordinates):
    view1 = coordinates
    view2 = np.random.permutation(coordinates)
    view3 = np.flip(coordinates, axis=1)
    return view1, view2, view3

After the views are generated, we'll write a function to train the model, which will contain our model architecture and hyperparameter definitions.

In [ ]:
def train_model(input_data, output_data):
    X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential()
    model.add(Dense(512, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='linear'))

    optimizer = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.1)

    loss = model.evaluate(X_test_scaled, y_test)
    print("Mean Squared Error on Test Data:", loss)

    return model, scaler

Now, given then the model, scaler, and the 3 views, we'll use the model to attempt to reconstruct the coordinates of the protein.

In [ ]:
def reconstruct_structure(model, scaler, views):
    reconstructed_views = []
    for view in views:
        scaled_view = scaler.transform(view)
        reconstructed_view = model.predict(scaled_view)
        reconstructed_views.append(reconstructed_view)
    return np.array(reconstructed_views)

Lastly, we'll write two methods to save the resulting PDB and visualize the structure in plotly.

In [ ]:
def save_pdb(output_path, coordinates):
    with open(output_path, 'w') as pdb_file:
        for i, coord in enumerate(coordinates):
            pdb_file.write(f"ATOM  {i+1:4}  CA  ALA A   1     {coord[0]:7.3f} {coord[1]:7.3f} {coord[2]:7.3f}  1.00  0.00           C\n")

def visualize_structure(coordinates, color_variable):
    fig = go.Figure()

    scatter = go.Scatter3d(
        x=coordinates[:, 0],
        y=coordinates[:, 1],
        z=coordinates[:, 2],
        mode='markers',
        marker=dict(size=5, color=color_variable, colorscale='Viridis', opacity=0.8),
        name='Protein Structure'
    )

    fig.add_trace(scatter)

    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ))

    fig.show()

Here's the main method to put it all together:

In [ ]:
def main():
    input_pdb_path = "input/AF-O06917-F1-model_v4.pdb"
    output_pdb_path = 'output/reconstructed_protein.pdb'
    original_coordinates = parse_pdb(input_pdb_path)

    view1, view2, view3 = generate_views(original_coordinates)

    input_data = np.vstack([view1, view2, view3])
    output_data = np.tile(original_coordinates, (3, 1))
    model, scaler = train_model(input_data, output_data)

    reconstructed_views = reconstruct_structure(model, scaler, [view1, view2, view3])

    color_variable = np.arange(len(np.mean(reconstructed_views, axis=0)))
    visualize_structure(np.mean(reconstructed_views, axis=0), color_variable)
    
    save_pdb(output_pdb_path, np.mean(reconstructed_views, axis=0))

if __name__ == "__main__":
    main()